1. For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

2. code to scrape the following Wikipedia page

In [136]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

#print(table)


In [148]:
res = []
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for tr in table_rows:
    td = tr.find_all("td")
    #print(td)
    
    for item in td:
        if "Not assigned" not in item.text: #Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
            #print("######")
            #print(item.text)
            #print(item.text.strip()[0:3])
            Postcode_u.append(item.text.strip()[0:3])
            if item.text.strip().find('(') != -1:
                #print(item.text.strip()[3:item.text.strip().find('(')])
                Borough_u.append(item.text.strip()[3:item.text.strip().find('(')])
                #print(item.text.strip()[item.text.strip().find('(')+1:item.text.strip().find(')')].replace(" / ",",").replace("/",","))
                Neighbourhood_u.append(item.text.strip()[item.text.strip().find('(')+1:item.text.strip().find(')')].replace(" / ",",").replace("/",",")) #More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table
            else:
                #print(item.text.strip()[3:len(item.text.strip())])
                Borough_u.append(item.text.strip()[3:len(item.text.strip())])
                Neighbourhood_u.append(item.text.strip()[3:len(item.text.strip())]) #If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
            #print("######")
        
    
canada_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
canada_df_m = pd.DataFrame.from_dict(canada_dict)
canada_df_m.to_csv('toronto_part1.csv')
canada_df_m.head(1000)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park\n\nOntario Provincial Government,Queen's Park\n\nOntario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [149]:
canada_df_m.shape

(103, 3)

In [ ]:
!pip install geocoder

In [79]:

import geocoder # import geocoder


def LL(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return (longitude,latitude)

3. As geocoder is not working considering longitude and latitude from URL https://raw.githubusercontent.com/symerio/postal-codes-data/master/data/geonames/CA.txt

In [164]:
postal_codes_data = pd.read_csv("https://raw.githubusercontent.com/symerio/postal-codes-data/master/data/geonames/CA.txt", sep="\t",header=None)
postal_codes_data.head(5)

postal_codes_data.columns

canada_geo_m = postal_codes_data[[1,9,10]]
canada_geo_m.columns = ["Postalcode", "Latitude" , "Longitude"]
canada_geo_m=canada_geo_m.loc[canada_geo_m["Postalcode"].str.startswith("M")]
canada_geo_m.head(5)

,Postalcode,Latitude,Longitude
671,M3K,43.7390,-79.4692
672,M4E,43.6784,-79.2941
673,M4J,43.6872,-79.3368
674,M4K,43.6803,-79.3538
675,M4L,43.6693,-79.3155


In [165]:
df_toronto_updated = pd.merge(canada_df_m, canada_geo_m, how='left', left_on = 'Postcode', right_on = 'Postalcode')
# remove the "Postal Code" column
df_toronto_updated.drop("Postalcode", axis=1, inplace=True)
df_toronto_updated.head(100)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park\n\nOntario Provincial Government,Queen's Park\n\nOntario Provincial Government,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern,Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


In [166]:
!pip install  folium
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [177]:
df_toronto_updated=df_toronto_updated.loc[df_toronto_updated["Borough"].str.contains("Toronto")]
df_toronto_updated.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6555,-79.3626
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
19,M4E,East Toronto,The Beaches,43.6784,-79.2941
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754


<h1>4. Explore and cluster the neighborhoods in Toronto</h1>

In [171]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [175]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
df_toronto=df_toronto_updated
# add markers to map

for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [179]:
df_toronto_denc = df_toronto.reset_index(drop=True)
df_toronto_denc.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6555,-79.3626
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
2,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
3,M4E,East Toronto,The Beaches,43.6784,-79.2941
4,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754


<h1> Explore First neighbourhood in our df using Foursquare </h1>

In [180]:

CLIENT_ID = '45NDOY2VY2Y0TAPY5JO30R3FG4LFZVFEZQFMF40WNPKNTSAP' # your Foursquare ID
CLIENT_SECRET = '50BBAWP15VB2B2ICCQYAQOQIM5PWE1EBAEAAMY31TMZ1KYPR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


neighborhood_name = df_toronto_denc.loc[0, 'Neighbourhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:
The first neighborhood's name is 'Regent Park,Harbourfront'.


In [181]:
neighborhood_latitude = df_toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park,Harbourfront are 43.6555, -79.3626.


In [182]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [183]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [184]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Souvlaki Express,Greek Restaurant,43.655584,-79.364438
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
5,Body Blitz Spa East,Spa,43.654735,-79.359874
6,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,Sukhothai,Thai Restaurant,43.658444,-79.365681
9,Sumach Espresso,Coffee Shop,43.658135,-79.359515


In [185]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [187]:
toronto_denc_venues = getNearbyVenues(names=df_toronto_denc['Neighbourhood'],
                                   latitudes=df_toronto_denc['Latitude'],
                                   longitudes=df_toronto_denc['Longitude']
                                  )

In [188]:
toronto_denc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park,Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park,Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park,Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park,Harbourfront",43.6555,-79.3626,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,"Regent Park,Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [189]:
toronto_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,93,93,93,93,93,93
"Brockton,Parkdale Village,Exhibition Place",39,39,39,39,39,39
"CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport",58,58,58,58,58,58
Central Bay Street,56,56,56,56,56,56
Christie,11,11,11,11,11,11
Church and Wellesley,76,76,76,76,76,76
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22
Davisville North,6,6,6,6,6,6


In [190]:
print('There are {} uniques categories.'.format(len(toronto_denc_venues['Venue Category'].unique())))

There are 212 uniques categories.


<h1> Analyze each Neighborhood </h1>

In [191]:
# one hot encoding
toronto_denc_onehot = pd.get_dummies(toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = toronto_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park,Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park,Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

<h1>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category </h1>

In [192]:
toronto_denc_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
toronto_denc_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.010753,0.000000,0.0,0.0,0.021505,0.000000,0.000000,0.000000,0.0,0.000000,0.010753,0.053763,0.000000,0.000000,0.0,0.010753,0.000000,0.032258,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.021505,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.043011,0.000000,0.000000,0.021505,0.0,0.000000,0.010753,0.032258,0.107527,0.0,0.000000,0.0,0.0,0.010753,0.000000,0.010753,0.0,0.0,0.0,0.010753,0.0,0.0,0.0,0.000000,0.021505,0.010753,0.0,0.010753,0.000000,0.0,0.0,0.0,0.000000,0.0,0.010753,0.000000,0.0,0.0,0.0,0.000000,0.021505,0.010753,0.0,0.010753,0.0,0.0,0.0,0.010753,0.010753,0.010753,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.010753,0.0,0.0,0.0,0.000000,0.0,0.010753,0.010753,0.010753,0.010753,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.043011,0.010753,0.0,0.0,0.0,0.0,0.010753,0.010753,0.032258,0.010753,0.0,0.010753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010753,0.010753,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.010753,0.0,0.0,0.0,0.01075

In [199]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Café,Seafood Restaurant,Hotel,Beer Bar,Restaurant,Cocktail Bar,Japanese Restaurant,Farmers Market
1,"Brockton,Parkdale Village,Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Chiropractor,Japanese Restaurant,Boutique,Brewery,Diner
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",Coffee Shop,Italian Restaurant,Bar,Café,Restaurant,Bakery,Gym / Fitness Center,Grocery Store,French Restaurant,Speakeasy
3,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Clothing Store,Spa,Japanese Restaurant,Fast Food Restaurant
4,Christie,Grocery Store,Café,Coffee Shop,Park,Playground,Candy Store,Baby Store,Wings Joint,Fast Food Restaurant,Farmers Market


<h1>Cluster neighborhoods</h1>

In [200]:

# Run K-Means clustring
# set number of clusters
kclusters = 5

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 0, 4, 4], dtype=int32)

In [201]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_denc_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6555,-79.3626,0,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Beer Store,Health Food Store,Wine Shop,Greek Restaurant,Italian Restaurant,Food Truck
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Movie Theater,Japanese Restaurant,Italian Restaurant,Café,Fast Food Restaurant
2,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Café,Coffee Shop,Seafood Restaurant,Gastropub,Italian Restaurant,Cocktail Bar,Park,Gym,Hotel,Restaurant
3,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Pub,Trail,Bakery,Gastropub,Health Food Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,0,Coffee Shop,Bakery,Café,Seafood Restaurant,Hotel,Beer Bar,Restaurant,Cocktail Bar,Japanese Restaurant,Farmers Market


In [203]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighbourhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [204]:
#cluster 1
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Beer Store,Health Food Store,Wine Shop,Greek Restaurant,Italian Restaurant,Food Truck
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Movie Theater,Japanese Restaurant,Italian Restaurant,Café,Fast Food Restaurant
2,Downtown Toronto,0,Café,Coffee Shop,Seafood Restaurant,Gastropub,Italian Restaurant,Cocktail Bar,Park,Gym,Hotel,Restaurant
3,East Toronto,0,Pub,Trail,Bakery,Gastropub,Health Food Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Donut Shop
4,Downtown Toronto,0,Coffee Shop,Bakery,Café,Seafood Restaurant,Hotel,Beer Bar,Restaurant,Cocktail Bar,Japanese Restaurant,Farmers Market
5,Downtown Toronto,0,Coffee Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Clothing Store,Spa,Japanese Restaurant,Fast Food Restaurant
7,Downtown Toronto,0,Café,Coffee Shop,Hotel,Restaurant,Gym,Salad Place,Asian Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant
11,West Toronto,0,Bar,Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Asian Restaurant,Yoga Studio,New American Restaurant,Seafood Restaurant,Café,Record Shop
12,East Toronto,0,Greek Restaurant,Ice Cream Shop,Café,Italian Restaurant,Coffee Shop,Yoga Studio,Dessert Shop,Bubble Tea Shop,Restaurant,Cocktail Bar
13,Downtown Toronto,0,Hotel,Coffee Shop,Café,Seafood Restaurant,Salad Place,Restaurant,Japanese Restaurant,Sandwich Place,Beer Bar,Breakfast Spot


In [205]:
#cluster 2
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East YorkEast Toronto,1,Convenience Store,Park,Wings Joint,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
18,Central Toronto,1,Photography Studio,Park,Wings Joint,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
22,West Toronto,1,Park,Residential Building (Apartment / Condo),Wings Joint,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [209]:
#cluster 3
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Spa,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [210]:
#cluster 4
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Pharmacy,Bus Line,Trail,Lawyer,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [211]:
#cluster 5
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Grocery Store,Café,Coffee Shop,Park,Playground,Candy Store,Baby Store,Wings Joint,Fast Food Restaurant,Farmers Market
8,West Toronto,4,Park,Bakery,Pool,Smoke Shop,Gym,Grocery Store,Pet Store,Pharmacy,Bus Line,Pizza Place
10,Downtown Toronto,4,Harbor / Marina,Park,Café,Music Venue,Wings Joint,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
20,Central Toronto,4,Breakfast Spot,Gym / Fitness Center,Dog Run,Park,Food & Drink Shop,Department Store,Deli / Bodega,Flower Shop,Fish Market,Fish & Chips Shop
23,Central Toronto,4,Playground,Gym Pool,Park,Garden,Wings Joint,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
29,Central Toronto,4,Thai Restaurant,Park,Gym,Grocery Store,Trail,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
33,Downtown Toronto,4,Playground,Grocery Store,Park,Candy Store,Wings Joint,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
